In [156]:
#loads the csv file into data frame
import pandas as pd

df = pd.read_csv('/home/pooja/projectACMS/train_lyrics_1511.csv')

df.head()

,artist,title,lyrics,mood,year
0,Casual,I Didn't Mean To,Verse One:\n\nAlright I might\nHave had a litt...,sad,1994.0
1,Adam Ant,Something Girls,Adam Ant/Marco Pirroni\nEvery girl is a someth...,happy,1982.0
2,Gob,Face the Ashes,"I've just erased it's been a while, I've got a...",sad,2007.0
3,Lionel Richie,Tonight Will Be Alright,Little darling \nWhere you've been so long \nI...,happy,1986.0
4,Blue Rodeo,Floating,"Lead Vocal by Greg\n\nWell, these late night c...",sad,1987.0


In [157]:
from sklearn.preprocessing import LabelEncoder
import pickle
import numpy as np

X_train = df['lyrics'].values 

y_train = df['mood'].values

print('before: %s ...' %y_train[:5])

le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)

print('after: %s ...' %y_train[:5])

before: ['sad' 'happy' 'sad' 'happy' 'sad'] ...
after: [1 0 1 0 1] ...


In [158]:
with open('/home/pooja/projectACMS/stopwords1.txt', 'r') as infile:
    stop_words1 = infile.read().splitlines()

In [159]:
import nltk
import string
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer


def Lemma_tokenizer(text, lemmatiser = WordNetLemmatizer()):
    
    lower_txt = text.lower()
    tokens = nltk.wordpunct_tokenize(lower_txt)
    stems = [lemmatiser.lemmatize(w, pos="v") for w in tokens]
    no_punct = [s for s in stems if re.match('^[a-zA-Z]+$', s) is not None]
    return no_punct

In [160]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(
            encoding='utf-8',
            decode_error='replace',
            strip_accents='unicode',
            analyzer='word',
            binary=False,
            stop_words=None,
            tokenizer=Lemma_tokenizer,
            ngram_range=(2,2)
    )

In [161]:
X_train_counts = count_vect.fit(X_train.ravel())
print('Vocabulary size: %s' %len(X_train_counts.get_feature_names()))

Vocabulary size: 95577


In [162]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
            encoding='utf-8',
            decode_error='replace',
            strip_accents='unicode',
            analyzer='word',
            binary=False,
            stop_words=None,
            tokenizer=Lemma_tokenizer,
            ngram_range=(2,2)
    )

In [163]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

TypeError: no supported conversion for types: (dtype('O'),)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf1 = MultinomialNB().fit(X_train_tfidf, y_train)


In [ ]:
import pandas as pd

df1 = pd.read_csv('/home/pooja/projectACMS/valid_lyrics_200.csv')

df1.head()

In [148]:
count=0
X_new_counts=[]
X_new_tdidf=[]
X_new_counts = count_vect.transform(df1['lyrics'].values)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf1.predict(X_new_tfidf)
#print(predicted)
for row_id in df1.index:
    mood=df1.loc[row_id]['mood']
    if mood=='happy':
        mood1=0
    else:
        mood1=1
    if (predicted[row_id])==mood1:
        count=count+1
print(count)    

103


In [149]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
text_clf1=Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', SVC()),])
text_clf1.fit(X_train, y_train)  
predicted = text_clf1.predict(df1['lyrics'].values)
count2=0
for row_id in df1.index:
    mood=df1.loc[row_id]['mood']
    if mood=='happy':
        mood1=0
    else:
        mood1=1
    if (predicted[row_id])==mood1:
        count2=count2+1
print(count2)

95


In [152]:
from sklearn.ensemble import RandomForestClassifier
text_clf2=Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', RandomForestClassifier() ),])
text_clf2.fit(X_train, y_train)  
predicted = text_clf2.predict(df1['lyrics'].values)
count3=0
for row_id in df1.index:
    mood=df1.loc[row_id]['mood']
    if mood=='happy':
        mood1=0
    else:
        mood1=1
    if (predicted[row_id])==mood1:
        count3=count3+1
print(count3)

125


In [153]:
print("acc of Random Forest =",count3/200)

acc of Random Forest = 0.625
